# balancing_areas and zone_balancing_areas

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###

import pandas as pd
import numpy as np

from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)
from pathlib import Path

import ast
import itertools
from statistics import mode

In [2]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

cwd = Path.cwd()

settings_path = (
    cwd / "settings_TD.yml" 
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("data_years")),
    end_year=max(settings.get("data_years")),
)

# check_settings(settings, pg_engine)

In [3]:
# read in csv outputs from PG
# existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
#new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
#existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
# potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
# fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
# fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
#load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [4]:
def balancing_areas(pudl_engine, IPM_regions, all_gen, quickstart_res_load_frac, quickstart_res_wind_frac, 
                         quickstart_res_solar_frac, spinning_res_load_frac, spinning_res_wind_frac, 
                         spinning_res_solar_frac):
    '''
    Function to create balancing_areas and zone_balancing_area tables
    Input:
        1) pudl_engine from init_pudl_connection
        2) IPM regions from settings.get('model_regions')
        3) all_gen pandas dataframe from gc.create_all_generators()
        4) quickstart_res_load_frac, quickstart_res_wind_frac, quickstart_res_solar_frac, 
            spinning_res_load_frac, spinning_res_wind_frac, and spinning_res_solar_frac:
            --> set these equal to values based on REAM
    Output:
        BALANCING_AREAS 
            * BALANCING_AREAS: based on balancing authority from pudl and connecting that to all_gen using plant_id_eia
            * other columns based on REAM Scenario 178
        ZONE_BALANCING_AREAS
            * Load_zone: IPM region
            * balancing_area
    '''
    # get table from PUDL that has  balancing_authority_code_eia
    plants_entity_eia = pd.read_sql_table("plants_entity_eia", pudl_engine)
    # dataframe with only balancing_authority_code_eia and plant_id_eia
    plants_entity_eia = plants_entity_eia[['balancing_authority_code_eia', 'plant_id_eia']]
    # create a dictionary that has plant_id_eia as key and the balancing authority as value
    plants_entity_eia_dict = plants_entity_eia.set_index('plant_id_eia').T.to_dict('list')
    
    plant_region_df = all_gen.copy()
    plant_region_df = plant_region_df[['plant_id_eia', 'region']]
    
    # get rid of NAs
    plant_region_df = plant_region_df[plant_region_df['plant_id_eia'].notna()]

    '''
    BALANCING_AREAS:
    take the plant_id_eia column from all_gen input, and return the balancing authority using 
        the PUDL plants_entity_eia dictionary
    
    '''
    # define function to get balancing_authority_code_eia from plant_id_eia
    def id_eia_to_bal_auth(plant_id_eia, plants_entity_eia_dict):
        if plant_id_eia in plants_entity_eia_dict.keys():
            return plants_entity_eia_dict[plant_id_eia][0] # get balancing_area from [balancing_area]
        else:
            return '-'

    # return balancing_authority_code_eia from PUDL table based on plant_id_eia
    plant_region_df['balancing_authority_code_eia'] = plant_region_df['plant_id_eia'].apply(
                lambda x: id_eia_to_bal_auth(x, plants_entity_eia_dict))
    
    # create output table
    balancing_areas = plant_region_df['balancing_authority_code_eia'].unique()
    BALANCING_AREAS = pd.DataFrame(balancing_areas, 
                                        columns=['BALANCING_AREAS'])
    BALANCING_AREAS['quickstart_res_load_frac'] = quickstart_res_load_frac
    BALANCING_AREAS['quickstart_res_wind_frac'] = quickstart_res_wind_frac
    BALANCING_AREAS['quickstart_res_solar_frac'] = quickstart_res_solar_frac
    BALANCING_AREAS['spinning_res_load_frac'] = spinning_res_load_frac
    BALANCING_AREAS['spinning_res_wind_frac'] = spinning_res_wind_frac
    BALANCING_AREAS['spinning_res_solar_frac'] = spinning_res_solar_frac
    
    '''
    ZONE_BALANCING_AREAS table:
        for each of the IPM regions, find the most common balancing_authority to create table
    '''
    
    zone_b_a_list = list()
    for ipm in IPM_regions:
        region_df = plant_region_df.loc[plant_region_df['region']==ipm]
        # take the most common balancing authority (assumption)
        bal_aut = mode(region_df['balancing_authority_code_eia'].to_list())
        zone_b_a_list.append([ipm, bal_aut])
    zone_b_a_list.append(['_ALL_ZONES', '.']) # Last line in the REAM inputs
    ZONE_BALANCING_AREAS = pd.DataFrame(zone_b_a_list, columns=['LOAD_ZONE', 'balancing_area'])
    
    return BALANCING_AREAS, ZONE_BALANCING_AREAS

In [5]:
IPM_regions = settings.get('model_regions')
bal_areas, zone_bal_areas = balancing_areas(pudl_engine, IPM_regions, all_gen, quickstart_res_load_frac=0.03, 
                          quickstart_res_wind_frac=0.05, quickstart_res_solar_frac=0.05, 
                          spinning_res_load_frac='.', spinning_res_wind_frac='.', spinning_res_solar_frac='.')

In [6]:
bal_areas

,BALANCING_AREAS,quickstart_res_load_frac,quickstart_res_wind_frac,quickstart_res_solar_frac,spinning_res_load_frac,spinning_res_wind_frac,spinning_res_solar_frac
0,AZPS,0.03,0.05,0.05,.,.,.
1,SRP,0.03,0.05,0.05,.,.,.
2,WALC,0.03,0.05,0.05,.,.,.
3,TEPC,0.03,0.05,0.05,.,.,.
4,GRIF,0.03,0.05,0.05,.,.,.
5,DEAA,0.03,0.05,0.05,.,.,.
6,HGMA,0.03,0.05,0.05,.,.,.
7,CISO,0.03,0.05,0.05,.,.,.
8,PSCO,0.03,0.05,0.05,.,.,.
9,WACM,0.03,0.05,0.05,.,.,.


In [7]:
# adding in the dummy loadzone for the fuel_cost / regional_fuel_market issue
zone_bal_areas.loc[len(zone_bal_areas.index)] = ['loadzone', 'BANC']
zone_bal_areas

,LOAD_ZONE,balancing_area
0,WEC_BANC,BANC
1,WEC_CALN,CISO
2,WEC_LADW,CISO
3,WEC_SDGE,CISO
4,WECC_AZ,AZPS
5,WECC_CO,PSCO
6,WECC_ID,IPCO
7,WECC_IID,IID
8,WECC_MT,NWMT
9,WECC_NM,PNM


In [8]:
bal_areas.to_csv(r'SWITCH_Inputs\balancing_areas.csv', index = False)
zone_bal_areas.to_csv(r'SWITCH_Inputs\zone_balancing_areas.csv', index = False)